In [1]:
import mido
import pandas as pd

# Function to convert MIDI note number to frequency
def midi_note_to_frequency(note_number):
    # MIDI note number to frequency conversion formula
    return 440.0 * (2.0 ** ((note_number - 69) / 12.0))

# a dictionary to store midi-conversion to key index
midi_note_to_key_index_dict = {
    0: 0,
    1: 20,
    2: 1,
    3: 21,
    4: 2,
    5: 3,
    6: 23,
    7: 4,
    8: 24,
    9: 5,
    10: 25,
    11: 6,
    12: 10,
    13: 30,
    14: 11,
    15: 31,
    16: 12,
    17: 13,
    18: 33,
    19: 14,
    20: 34,
    21: 15,
    22: 35,
    23: 16,
    24: 17,
    25: 37,
    26: 18,
    27: 38,
    28: 19
}

def midi_note_to_key_index(note_number):
    # MIDI note number to piano display coordinates conversion formula
    #
    # Because the piano display starts at C3 and goes through E5, which 
    # is not an integer multiple of an octave above C3, the conversion 
    # only maps to the 12 keys from C3 (played with z) to B3 (played with m)
    # and the 12 keys from C4 (played with q) to B4 (played with u)
    # 
    # The conversion is done with modulo arithmatic
    #
    # The entire midi note range, from 0 to 127, is mapped in two octave 
    # chunks to 24 keys. 
    note_number = (note_number - 60) % 24
    return(midi_note_to_key_index_dict[note_number])

# values from the 01_SineEnv_Piano.cpp file from the allolib_playground tutorials folder
keyWidth = 116
keyHeight = 296
keyPadding = 2

# translated from the 01_SineEnv_Piano.cpp file from the allolib_playground tutorials folder
def key_index_to_coordinates(key_index):
    is_black_key = False
    if(key_index >= 20):
        key_index -= 20
        is_black_key = True
        
    w = keyWidth
    h = keyHeight
    x = (keyWidth + keyPadding * 2) * (key_index % 10) + keyPadding
    y = 0
    
    if(is_black_key == True):
        x += keyWidth * 0.5
        y += keyHeight * 0.5
        h *= 0.5
        
    if(key_index >= 10):
        y += keyHeight + keyPadding * 2
    
    return (w,h,x,y)

# Load MIDI file
midi_file = mido.MidiFile('dafengchui.mid')

# Create a list to hold note information
notes_list = []

# Initialize a variable to keep track of the current time
current_time = 0

# Dictionary to keep track of note start times
note_start_times = {}

# Dictionary to keep track of note key coordinates
note_key_coordinates = {}

# offset for program start delay
offset = 0.1

# Parameters for the SineEnv Synthesizer
amplitude = 0.2
attack_time = 0.01
release_time = 0.1
pan = 0

# Iterate over all messages in the MIDI file
for msg in midi_file:
    # Update the current time
    current_time += msg.time
    if msg.type == 'note_on' and msg.velocity > 0:
        # Note on message, start of a note
        note_start_times[msg.note] = current_time
        # Store the key coordinates for the note
        note_key_coordinates[msg.note] = key_index_to_coordinates(midi_note_to_key_index(msg.note))
    elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
        # Note off message, end of a note
        if msg.note in note_start_times:
            note_start_time = note_start_times.pop(msg.note)
            note_end_time = current_time
            frequency = midi_note_to_frequency(msg.note)
            key_coordinates = note_key_coordinates.pop(msg.note)
            #print(msg.note, ' ', key_coordinates)
            notes_list.append(['@', note_start_time + 0.1, note_end_time + 0.1, 'SineEnv', amplitude, frequency, attack_time, release_time, pan, key_coordinates[2], key_coordinates[3], key_coordinates[0], key_coordinates[1]])

# Create a DataFrame with the notes list
df = pd.DataFrame(notes_list, columns=[' ', 'start_time', 'end_time', 'SineEnv', 'amplitude', 'frequency', 'attack_time', 'release_time', 'pan', 'pianoKeyX', 'pianoKeyY', 'pianoKeyWidth', 'pianoKeyHeight'])

# Display the DataFrame
# print(df)


In [2]:
df

,,start_time,end_time,SineEnv,amplitude,frequency,attack_time,release_time,pan,pianoKeyX,pianoKeyY,pianoKeyWidth,pianoKeyHeight
0,@,0.100000,0.668333,SineEnv,0.2,246.941651,0.01,0.1,0,722.0,300.0,116,296.0
1,@,0.100000,0.668333,SineEnv,0.2,293.664768,0.01,0.1,0,122.0,0.0,116,296.0
2,@,0.100000,0.668333,SineEnv,0.2,369.994423,0.01,0.1,0,420.0,148.0,116,148.0
3,@,0.700000,0.888333,SineEnv,0.2,246.941651,0.01,0.1,0,722.0,300.0,116,296.0
4,@,0.700000,0.888333,SineEnv,0.2,293.664768,0.01,0.1,0,122.0,0.0,116,296.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1702,@,262.100000,262.478333,SineEnv,0.2,184.997211,0.01,0.1,0,420.0,448.0,116,148.0
1703,@,262.563333,264.896667,SineEnv,0.2,164.813778,0.01,0.1,0,242.0,300.0,116,296.0
1704,@,262.631667,264.896667,SineEnv,0.2,220.000000,0.01,0.1,0,602.0,300.0,116,296.0
1705,@,262.698333,264.896667,SineEnv,0.2,277.182631,0.01,0.1,0,60.0,148.0,116,148.0


In [3]:
# replace end_time with duration without changing the order of the columns
duration = df['end_time'] - df['start_time']
df = df.drop(columns=['end_time'])
df.insert(2, 'duration', duration)

df

,,start_time,duration,SineEnv,amplitude,frequency,attack_time,release_time,pan,pianoKeyX,pianoKeyY,pianoKeyWidth,pianoKeyHeight
0,@,0.100000,0.568333,SineEnv,0.2,246.941651,0.01,0.1,0,722.0,300.0,116,296.0
1,@,0.100000,0.568333,SineEnv,0.2,293.664768,0.01,0.1,0,122.0,0.0,116,296.0
2,@,0.100000,0.568333,SineEnv,0.2,369.994423,0.01,0.1,0,420.0,148.0,116,148.0
3,@,0.700000,0.188333,SineEnv,0.2,246.941651,0.01,0.1,0,722.0,300.0,116,296.0
4,@,0.700000,0.188333,SineEnv,0.2,293.664768,0.01,0.1,0,122.0,0.0,116,296.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1702,@,262.100000,0.378333,SineEnv,0.2,184.997211,0.01,0.1,0,420.0,448.0,116,148.0
1703,@,262.563333,2.333333,SineEnv,0.2,164.813778,0.01,0.1,0,242.0,300.0,116,296.0
1704,@,262.631667,2.265000,SineEnv,0.2,220.000000,0.01,0.1,0,602.0,300.0,116,296.0
1705,@,262.698333,2.198333,SineEnv,0.2,277.182631,0.01,0.1,0,60.0,148.0,116,148.0


In [4]:
# convert dataframe to a space separated file
df.to_csv('bin/SineEnv_Piano-data/dafengchui.synthSequence', sep=' ', index=False, header=False)